# Assignment #7: Demonstrate Knapsack via Multiple Techniques

---


Karim Kanji  <br>
IA-20<br>
13.12.2023<br>

## 1. Exact Optimization Technique (Dynamic Programming)

In [1]:
def knapsackDP(values, weights, sizes, W, S):
    # DP initialization
    n = len(values)
    dp = [[[0 for _ in range(S + 1)] for _ in range(W + 1)] for _ in range(n + 1)]

    # DP computation
    for i in range(1, n + 1):
        for w in range(1, W + 1):
            for s in range(1, S + 1):
                if weights[i - 1] <= w and sizes[i - 1] <= s:
                    dp[i][w][s] = max(values[i - 1] + dp[i - 1][w - weights[i - 1]][s - sizes[i - 1]], dp[i - 1][w][s])
                else:
                    dp[i][w][s] = dp[i - 1][w][s]
    return dp[n][W][S]




---



## 2. Genetic Algorithm

In [6]:
import random

def fitness(individual, values, weights, sizes, W, S):
    total_value, total_weight, total_size = 0, 0, 0
    for i in range(len(individual)):
        if individual[i] == 1:
            total_value += values[i]
            total_weight += weights[i]
            total_size += sizes[i]
    if total_weight > W or total_size > S:
        return 0
    return total_value

def initialize_population(n_items, population_size):
    return [[random.randint(0, 1) for _ in range(n_items)] for _ in range(population_size)]

def select_parents(population, fitness_scores):
    parents = random.choices(population, weights=fitness_scores, k=2)
    return parents[0], parents[1]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_rate=0.05):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

def geneticAlgorithm(values, weights, sizes, W, S, population_size=50, generations=100):
    n_items = len(values)
    population = initialize_population(n_items, population_size)

    for generation in range(generations):
        fitness_scores = [fitness(individual, values, weights, sizes, W, S) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])

        population = new_population

    best_individual = max(population, key=lambda individual: fitness(individual, values, weights, sizes, W, S))
    return best_individual, fitness(best_individual, values, weights, sizes, W, S)

# Example usage
values = [60, 100, 120]
weights = [10, 20, 30]
sizes = [2, 3, 4]
W = 50
S = 5
result = geneticAlgorithm(values, weights, sizes, W, S)
print("Best Individual:", result[0])
print("Best Fitness:", result[1])

Best Individual: [1, 1, 0]
Best Fitness: 160




---



## 3. Simulated Annealing

In [7]:
import math
import random

def fitness(individual, values, weights, sizes, W, S):
    total_value, total_weight, total_size = 0, 0, 0
    for i in range(len(individual)):
        if individual[i] == 1:
            total_value += values[i]
            total_weight += weights[i]
            total_size += sizes[i]
    if total_weight > W or total_size > S:
        return 0
    return total_value

def random_solution(n_items):
    return [random.randint(0, 1) for _ in range(n_items)]

def get_neighbor(solution):
    neighbor = solution.copy()
    position_to_flip = random.randint(0, len(neighbor) - 1)
    neighbor[position_to_flip] = 1 - neighbor[position_to_flip]
    return neighbor

def simulatedAnnealing(values, weights, sizes, W, S, T=1.0, T_min=0.00001, alpha=0.9):
    current_solution = random_solution(len(values))
    current_fitness = fitness(current_solution, values, weights, sizes, W, S)

    while T > T_min:
        for _ in range(100):  # Iterations at each temperature level
            next_solution = get_neighbor(current_solution)
            next_fitness = fitness(next_solution, values, weights, sizes, W, S)

            acceptance_probability = math.exp((next_fitness - current_fitness) / T)
            if next_fitness > current_fitness or random.random() < acceptance_probability:
                current_solution = next_solution
                current_fitness = next_fitness

        T = T * alpha

    return current_solution, current_fitness

# Example usage
values = [60, 100, 120]
weights = [10, 20, 30]
sizes = [2, 3, 4]
W = 50
S = 5
result = simulatedAnnealing(values, weights, sizes, W, S)
print("Best Individual:", result[0])
print("Best Fitness:", result[1])


Best Individual: [1, 1, 0]
Best Fitness: 160




---



## 4. Constructive Heuristic Approach

In [9]:
def heuristicKnapsack(values, weights, sizes, W, S):
    # Create a list of items with their value-to-weight-plus-size ratio and index
    ratio = [(values[i] / max(weights[i] + sizes[i], 1), i) for i in range(len(values))]

    # Sort items based on this ratio in descending order
    ratio.sort(reverse=True)

    total_weight, total_size, total_value = 0, 0, 0
    selected_items = []

    # Iterate over items and select the best ones based on the available capacity
    for r, i in ratio:
        if total_weight + weights[i] <= W and total_size + sizes[i] <= S:
            total_weight += weights[i]
            total_size += sizes[i]
            total_value += values[i]
            selected_items.append(i)

    return total_value, selected_items

# Example usage
values = [60, 100, 120]
weights = [10, 20, 30]
sizes = [2, 3, 4]
W = 50
S = 5
result = heuristicKnapsack(values, weights, sizes, W, S)
print("Total Value:", result[0])
print("Selected Items:", result[1])


Total Value: 160
Selected Items: [0, 1]
